# Machine Learning II: Assignments 2
Leon Berghoff, Jiawei Li, Strahinja Trenkic, Otto Riess

In [1]:
import pandas as pd
from bank_mkt import import_dataset, split_dataset, transform
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

# Data Preparation

In [7]:
bank_mkt = import_dataset("BankMarketing.csv")

func_transformer = FunctionTransformer(transform, kw_args={"gen": ["days", "has_previous"]})

num_features = ["month",
                "day_of_week",
                "campaign",
                "previous",
                "emp.var.rate",
                "cons.price.idx",
                "cons.conf.idx",
                "euribor3m",
                "nr.employed"]
standard_scaler = ColumnTransformer([("scaler", StandardScaler(), num_features)], remainder="passthrough")

preprocessor = make_pipeline(func_transformer, standard_scaler)

X_train, y_train, X_test, y_test, *_ = split_dataset(bank_mkt, preprocessor)